In [38]:
import numpy as np
import pandas as pd
from scipy.sparse import bsr_matrix
import scipy

In [2]:
# user_embeddings = np.load("user_embeddings.npy")
# item_embeddings = np.load("item_embeddings.npy")

In [89]:
data = pd.read_csv('data/ratings.csv',names=["User","Anime","Rating"])

In [59]:
data = data[data.User < 20000]

In [90]:
data = data.groupby("User").filter(lambda x: len(x) > 300)

In [91]:
len(data.User.unique())

4418

In [74]:
ratings = bsr_matrix((data["Rating"], (data["Anime"], data["User"]))).toarray()

In [49]:
print(ratings)

[[0 0 0 ... 0 0 0]
 [0 8 0 ... 0 0 0]
 [0 7 0 ... 8 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [50]:
#ratings = np.matmul(user_embeddings,item_embeddings.T)

In [51]:
#print(ratings.shape)

In [52]:
def compute_similarity(user_ratings,ratings):
    
    idx = np.where(user_ratings != 0)
    p_avg = np.average(ratings,axis=1)
    u_avg = np.average(user_ratings)
    rated = ratings[:,idx[0]]
    user_rated = user_ratings[idx[0]]
    norm_rated = np.subtract(rated.T,p_avg)
    norm_u = np.subtract(user_rated.T,u_avg)
    sum_p_u = np.matmul(norm_rated.T,norm_u.T)
    sum_p = np.power(norm_rated,2)
    sum_u = np.power(norm_u,2)
    dist = np.sqrt(np.sum(sum_p)*np.sum(sum_u))
    sim = np.divide(sum_p_u,dist)
    return sim

def predict_ratings(user_ratings,ratings,sim):
    #sim = compute_similarity(user_ratings,ratings)
    idx = np.where(user_ratings == 0)
    non_idx = np.where(user_ratings != 0)
    avg = np.average(user_ratings[non_idx])
    #print(idx)
    user_rated = ratings[:,idx[0]]
    #print(user_rated.shape)
    predicted_ratings = np.zeros(user_rated.shape[1])
    for i in np.arange(user_rated.shape[1]):
        rated_idx = np.where(user_rated[:,i] != 0)
        sim_rated = sim[rated_idx]
        sim_rated = sim_rated.reshape((sim_rated.shape[0],1)).T
        rated = user_rated[rated_idx,i].T
        temp = np.matmul(sim_rated,rated)/sim_rated.shape[1]
        predicted_ratings[i] = temp + avg
    return predicted_ratings,idx

    #print(rated.shape)

In [53]:
#test_rating = np.random.randint(0,11,size=item_embeddings.shape[0])
test_rating = np.zeros(ratings.shape[0])
for i in range(0,100):
    test_rating[i] = 10
print(test_rating.shape)

(3539,)


In [54]:
def topNAnime(user_ratings,ratings,n=10):
    sim = compute_similarity(user_ratings,ratings)
    predicted_ratings,idx = predict_ratings(user_ratings,ratings,sim)
    old_idx = np.where(user_ratings != 0)
    rating_tuple = [(predicted_ratings[i],idx[0][i]) for i in np.arange(predicted_ratings.shape[0])]
    sorted_ratings = sorted(rating_tuple, key = lambda tup: tup[0],reverse=True)
    topN = [i[1] for i in sorted_ratings[:n]]
    return topN

In [55]:
sorted_ratings = topNAnime(test_rating,ratings)

In [56]:
print(sorted_ratings)

[2393, 676, 2505, 3424, 2772, 2218, 537, 3500, 2863, 1471]


In [92]:
scipy.sparse.save_npz('ratings.npz', bsr_matrix((data["Rating"], (data["Anime"], data["User"]))))